In [1]:
!pip install -qU \
    langchain==0.0.354 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2 \
    google-generativeai==0.3.1 \
    google-ai-generativelanguage==0.4.0 \
    langchain-google-genai \
    sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import google.generativeai as genai
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage
#libraries for google drive authentication
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.schema import Document
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

In [3]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GeminiAPIKey')

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

### **Building RAG**

In [4]:
#authenticate with you google drive credentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
loader = TextLoader("/content/drive/MyDrive/ADHDFAQ.txt")
documents = loader.load()

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

In [9]:
docs[:3]

[Document(page_content='Q: What is ADHD?\nA: ADHD stands for Attention Deficit Hyperactivity Disorder. ADHD  is a neuro-developmental disorder characterised by symptoms of inattention, hyperactivity, and impulsivity. ADHD is a chronic condition that impacts an individual’s emotions, behaviours, and the ability to learn new things.', metadata={'source': '/content/drive/MyDrive/ADHDFAQ.txt'}),
 Document(page_content='Q: What are the main types of ADHD?\nA: There are three main types of ADHD:\n\t•\tPredominantly Inattentive Presentation\n\t•\tPredominantly Hyperactive-Impulsive Presentation\n\t•\tCombined Presentation\nEveryone is different, so it’s common for two people to experience the same symptoms in different ways. For example, these behaviours are often different in boys and girls. Boys may be seen as more hyperactive, and girls may be quietly inattentive. If you want to know more about each type please ask.', metadata={'source': '/content/drive/MyDrive/ADHDFAQ.txt'}),
 Document(pa

In [10]:
# Extract questions and answers
questions = []
qna_pairs = []

for doc in docs:
    if doc.page_content.startswith('Q:'):
        questions.append(doc.page_content.split('\n')[0])
        qna_pairs.append(doc)

### **Embedding Model**

Using Huggling Face transformer to embed text into vectors

In [16]:
# embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embedding_model = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedded with a dimension of 1024

### **Vector Database Setup**

Using Pinecone

In [12]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
# initialize connection to pinecone (get API key at https://www.pinecone.io/)
api_key = userdata.get('test')

# configure client
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1")

In [ ]:
# import time

# index_name = 'adhd-q-only'

# existing_indexes = [
#     index_info["name"] for index_info in pc.list_indexes()
# ]

# # check if index already exists (it shouldn't if this is first time)
# if index_name not in existing_indexes:
#     # if does not exist, create index
#     pc.create_index(
#         index_name,
#         dimension=1024,  # dimensionality of our embedding model
#         metric='cosine',
#         spec=spec
#     )
#     # wait for index to be initialized
#     while not pc.describe_index(index_name).status['ready']:
#         time.sleep(1)

# # connect to index
# index = pc.Index(index_name)
# time.sleep(1)
# # view index stats
# index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
# import time

# index_name_qa = 'adhd-qa'

# existing_indexes = [
#     index_info["name"] for index_info in pc.list_indexes()
# ]

# # check if index already exists (it shouldn't if this is first time)
# if index_name_qa not in existing_indexes:
#     # if does not exist, create index
#     pc.create_index(
#         index_name_qa,
#         dimension=1024,  # dimensionality of our embedding model
#         metric='cosine',
#         spec=spec
#     )
#     # wait for index to be initialized
#     while not pc.describe_index(index_name_qa).status['ready']:
#         time.sleep(1)

# # connect to index
# index_qa = pc.Index(index_name_qa)
# time.sleep(1)
# # view index stats
# index_qa.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 34}},
 'total_vector_count': 34}

In [14]:
import time

index_name_qa2 = 'adhd-qa2'

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name_qa2 not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name_qa2,
        dimension=1024,  # dimensionality of our embedding model
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name_qa2).status['ready']:
        time.sleep(1)

# connect to index
index_qa2 = pc.Index(index_name_qa2)
time.sleep(1)
# view index stats
index_qa2.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 34}},
 'total_vector_count': 34}

storing our docs from doc as embeddings into the vector

In [ ]:
# from tqdm.auto import tqdm  # for progress bar

# # data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

# for i in tqdm(range(0, len(docs))):
#     # generate unique ids for each chunk
#     id = str(i)
#     text = questions[i]
#     # get text to embed
#     embeds = embedding_model.embed_documents([text])
#     # get metadata to store in Pinecone
#     metadata = {'text': text, 'source': qna_pairs[i].metadata["source"], 'index': id}
#     # add to Pinecone
#     index.upsert(vectors=[(id, embeds[0], metadata)])

  0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
# from tqdm.auto import tqdm

# for i in tqdm(range(0, len(docs))):
#     # generate unique ids for each chunk
#     id = [str(i)]
#     texts = docs[i].page_content
#     embeds = embedding_model.embed_documents(texts)
#     # get metadata to store in Pinecone
#     metadata = [
#         {'text': texts,
#          'source': docs[i].metadata["source"]}
#     ]
#     # add to Pinecone
#     index_qa.upsert(vectors=zip(id, embeds, metadata))

  0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
# from tqdm.auto import tqdm  # for progress bar

# # data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

# for i in tqdm(range(0, len(docs))):
#     # generate unique ids for each chunk
#     id = str(i)
#     text = docs[i].page_content
#     # get text to embed
#     embeds = embedding_model.embed_documents([text])
#     # get metadata to store in Pinecone
#     metadata = {'text': text, 'source': qna_pairs[i].metadata["source"], 'index': id}
#     # add to Pinecone
#     index_qa2.upsert(vectors=[(id, embeds[0], metadata)])

  0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
index_qa2.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 34}},
 'total_vector_count': 34}

Now we have created the Vector database we will create vector store so we can use the index/database to use locally in our notebook so we can do a similarily search / retreival of relevant documents for context

In [19]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text
vectorstore_qa2 = Pinecone(index_qa2, embedding_model.embed_query, text_field)

Testing retrival and embedding

In [20]:
query = 'what is your email'
vectorstore_qa2.similarity_search(query, k=3)

[Document(page_content='Q: How do I book an appointment?\nA: You can book an appointment by calling us at 04 444 4444, emailing us at info@adhdspecialistsaustralia.com, or using our online booking system on our website www.adhdspecialistsaustralia.com.', metadata={'index': '13', 'source': './ADHDFAQ.txt'}),
 Document(page_content='Q: What are the steps to get an ADHD diagnosis?\nA: 1. Referral: Obtain a referral from your GP or another healthcare provider. This is required for booking an appointment with our specialists. 2. Initial Contact:Contact our clinic by phone at 04 444 4444 or email at info@adhdspecialistsaustralia.com or using our online booking system on our website www.adhdspecialistsaustralia.com to express your interest in getting an ADHD evaluation. Our administrative staff will guide you through the initial steps and help you set up an appointment. 3. Pre-Appointment Paperwork:Complete the necessary pre-appointment paperwork, including medical history forms and symptom q

Function that will tak the query as input, process top 3 results, join it to 1 single string.
It will take the query input and embed it into a vector with huggingface transformer, do a similarity search using cosine against the vectors in the Pinecone vectorestore and return the top 3 most similar texts that are stored as the metadata. This will be stored as source knowledge in the context list to generate AI responses.

In [ ]:
# def augment_prompt(query: str):
#     # get top 3 results from knowledge base
#     results = vectorstore.similarity_search(query, k=3)

#     # get the text from the results
#     # source_knowledge = "\n".join([x.page_content for x in results])
#     source_knowledge = "\n".join([qna_pairs[int(x.metadata['index'])].page_content for x in results])

#     # feed into an augmented prompt
#     augmented_prompt = f"""Using the contexts below, answer the query.

#     Contexts:
#     {source_knowledge}

#     Query: {query}"""
#     return augmented_prompt

In [21]:
def augment_prompt_qa(query: str):
    # get top 3 results from knowledge base
    results = vectorstore_qa2.similarity_search(query, k=3)

    # get the text from the results
    # source_knowledge = "\n".join([x.page_content for x in results])
    source_knowledge = "\n".join([x.page_content for x in results])

    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

#### **ChatBot Model Parameters**

For a chatbot providing medical and factual information regarding ADHD at a specialist clinic, you want the responses to be reliable and precise. Therefore, a lower temperature and top-p are advisable to ensure consistency and accuracy.

In [22]:
model = ChatGoogleGenerativeAI(model="models/gemini-1.0-pro-latest",
                             temperature=0.2, top_p=0.1)

Now creating the workflow for our chatbot

In [23]:
#system message

context = [HumanMessage(content = 'Your role is an assistant for a \
ADHD specialist clinic called ADHD Specialists Australia \
to answer questions people have regarding the clinic and ADHD. \
You first welcome me and ask how you can help me.')]
response = model(context)
chat_history = [response]
context.append(response)
display(Markdown(response.content))

while True:
  query = input('Your query: ')
  if query.lower() in ['exit', 'end', 'bye', 'quit', 'goodbye']:
    break

  human_message = HumanMessage(content=query)
  new_context =  HumanMessage(content=augment_prompt_qa(query))
  context.append(new_context)
  context.append(human_message)
  chat_history.append(human_message)

  response = model(context)
  context.append(response)
  chat_history.append(response)
  display(Markdown(response.content))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


**Welcome to ADHD Specialists Australia!**

I'm here to assist you with any questions you may have about our clinic and ADHD. How can I help you today?

Your query: What is your email


Our email address is **info@adhdspecialistsaustralia.com**.

Your query: do you do assessments


Yes, we do assessments for ADHD. Our assessment process includes a comprehensive evaluation with questionnaires, behavioral assessments, and possibly additional tests to diagnose ADHD accurately.

The assessment process can take several weeks, depending on the complexity of the case and the availability of appointments.

To start the assessment process, you can book an appointment by calling us at 04 444 4444, emailing us at info@adhdspecialistsaustralia.com, or using our online booking system on our website www.adhdspecialistsaustralia.com.

Your query: what is the best way to make an appointment


The best way to make an appointment is to use our online booking system on our website www.adhdspecialistsaustralia.com. This system allows you to view our availability and book an appointment at a time that is convenient for you.

You can also book an appointment by calling us at 04 444 4444 or emailing us at info@adhdspecialistsaustralia.com. However, we recommend using the online booking system as it is the most efficient way to schedule an appointment.

Please note that we require a referral from your GP or another healthcare provider before you can book an appointment with our specialists.

Your query: end


In [24]:
#obtaining the chat history if you want later
for i in range(len(chat_history)):
  display(Markdown(chat_history[i].content))

**Welcome to ADHD Specialists Australia!**

I'm here to assist you with any questions you may have about our clinic and ADHD. How can I help you today?

What is your email

Our email address is **info@adhdspecialistsaustralia.com**.

do you do assessments

Yes, we do assessments for ADHD. Our assessment process includes a comprehensive evaluation with questionnaires, behavioral assessments, and possibly additional tests to diagnose ADHD accurately.

The assessment process can take several weeks, depending on the complexity of the case and the availability of appointments.

To start the assessment process, you can book an appointment by calling us at 04 444 4444, emailing us at info@adhdspecialistsaustralia.com, or using our online booking system on our website www.adhdspecialistsaustralia.com.

what is the best way to make an appointment

The best way to make an appointment is to use our online booking system on our website www.adhdspecialistsaustralia.com. This system allows you to view our availability and book an appointment at a time that is convenient for you.

You can also book an appointment by calling us at 04 444 4444 or emailing us at info@adhdspecialistsaustralia.com. However, we recommend using the online booking system as it is the most efficient way to schedule an appointment.

Please note that we require a referral from your GP or another healthcare provider before you can book an appointment with our specialists.

In [25]:
context

[HumanMessage(content='Your role is an assistant for a ADHD specialist clinic called ADHD Specialists Australia to answer questions people have regarding the clinic and ADHD. You first welcome me and ask how you can help me.'),
 AIMessage(content="**Welcome to ADHD Specialists Australia!**\n\nI'm here to assist you with any questions you may have about our clinic and ADHD. How can I help you today?"),
 HumanMessage(content='Using the contexts below, answer the query.\n\n    Contexts:\n    Q: How do I book an appointment?\nA: You can book an appointment by calling us at 04 444 4444, emailing us at info@adhdspecialistsaustralia.com, or using our online booking system on our website www.adhdspecialistsaustralia.com.\nQ: What are the steps to get an ADHD diagnosis?\nA: 1. Referral: Obtain a referral from your GP or another healthcare provider. This is required for booking an appointment with our specialists. 2. Initial Contact:Contact our clinic by phone at 04 444 4444 or email at info@adh